# 서울스토어 크롤러2

분류용 상품 이미지를 크롤링하기 위한 메타 정보 크롤러입니다. a.k.a "통" 만들기 사전작업

서울스토어의 카테고리별 상품 정보(이미지, 상세정보 링크)를 크롤링하는 크롤러입니다. 크롤링 결과를 활용해서 상품 메타정보를 저장 + 상품 이미지(별도의 크롤러 활용)를 저장합니다.

크롤링한 카테고리는 아래와 같습니다.
- 상의
- 하의
- 원피스

크롤링 결과는 {"key": 고유번호(5자리수), "product_url": 상품url, "img_url": 이미지url} 형태의 json 파일로 저장합니다.

In [1]:
from __future__ import unicode_literals
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import glob
import json
import os
import re
import sys
import time
import traceback
from builtins import open
from time import sleep

In [2]:
#카테고리별 url(상의, 하의, 원피스)
root_urls = ['https://www.seoulstore.com/categories/1251/regDatetime/desc',
            'https://www.seoulstore.com/categories/1253/regDatetime/desc',
            'https://www.seoulstore.com/categories/1252/regDatetime/desc']

In [17]:
def get_products(urls, num, wait_time):
    
    """
    Arguments:
    - urls: 리스트. 크롤링할 url들을 원소로 함.
    - num: url당 크롤링할 아이템 수
    - wait_time: 로딩을 몇 초까지 기다릴지
    
    Return:
    - total_posts: 리스트. 아이템 당 {"product_url": 상품url, "img_url": 이미지url} 정보를 크롤링한 것을 원소로 함.
    
    크롤링 완료 후 "key":고유번호(5자리수)를 추가해줄 것임 (key를 사후에 추가하는 이유는 중복 크롤링 처리를 쉽게 하기 위함임)
    """
    
    total_posts = []  #전체 크롤링 결과를 저장할 리스트
    product_set = set()   #중복 크롤링 거르기 위한 셋. product_url을 원소로 함
    
    browser = webdriver.Chrome('chromedriver')   #크롬 브라우저 실행
    wait = WebDriverWait(browser, wait_time)
    
    for url in urls:   #인자로 주어진 url 하나씩 취함
        
        posts = []   #당해 url 크롤링 결과를 저장할 리스트
        
        browser.get(url)
        body = browser.find_element_by_tag_name('body')

        count = 0    #더 이상 로드되는 데이터가 없을 시 크롤링 종료하기 위해 필요한 count임
        prev_posts_count = 0

        while len(posts) < num:

            body.send_keys(Keys.PAGE_DOWN)
            ele_posts = browser.find_elements_by_class_name('image_container')
            cur_posts_count = len(ele_posts)

            if prev_posts_count == cur_posts_count:
                count += 1

            else:   #로드된 데이터가 더 있다면 count를 0으로 초기화
                count = 0

            if count == 50:  #스크롤을 50회 이상했는데도 추가로 로드되는 데이터가 더 이상 없을 시 break
                break

            for ele in ele_posts:
                product_url= ele.find_element_by_tag_name('a').get_attribute('href')
                if product_url not in product_set:
                    dict_post = { "product_url": product_url }
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
                    ele_img = ele.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
                    dict_post["img_url"] = ele_img.get_attribute("src")
                    product_set.add(product_url)
                    posts.append(dict_post)

            prev_posts_count = cur_posts_count

        total_posts.extend(posts[:num])
        
    return total_posts
        
def output(data, filepath):
    out = json.dumps(data, ensure_ascii=False)
    if filepath:
        with open(filepath, "w", encoding="utf8") as f:
            f.write(out)
    else:
        print(out)

In [21]:
output(get_products(root_urls, 3000, 300), '../tong.json')

AttributeError: 'NoneType' object has no attribute 'get_attribute'

In [19]:
with open('../tong.json') as data_file:
    data = json.load(data_file)

In [20]:
len(data)

30